In [ ]:
!nvidia-smi

Mon Oct  4 04:51:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import tensorflow as tf


def get_angles(position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

def positional_encoding(position, d_model):
    #tf.newaxis : expand dimensions
    angle_rads = get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # list[<start>:<end>:<step>] even indices '0::2'
    sines = tf.math.sin(angle_rads[:, 0::2])

    # list[<start>:<end>:<step>] odd indices '1::2'
    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


def create_look_ahead_mask(size):
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return look_ahead_mask

def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead)
    but it must be broadcastable for addition.
    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.
    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights


class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff,
               input_vocab_size, maximum_position_encoding, dropout_rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, self.d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, dropout_rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)


class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff,
               target_vocab_size, maximum_position_encoding, dropout_rate=0.1):

    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, dropout_rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights


class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff,
               input_vocab_size,target_vocab_size,
               positional_encoding_input,positional_encoding_target,
               dropout_rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size,positional_encoding_input, dropout_rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                           target_vocab_size, positional_encoding_target, dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, input, target, training, enc_padding_mask,
           look_ahead_mask, dec_padding_mask):

    enc_output = self.encoder(input, training, enc_padding_mask)  # (batch_size, input_seq_len, d_model)

    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        target, enc_output, training, look_ahead_mask, dec_padding_mask)

    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

    return enc_output, dec_output, final_output, attention_weights

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
class Encoder2(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff,
               intensity_vocab_size, dropout_rate=0.1):
    super(Encoder2, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(intensity_vocab_size, self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, dropout_rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  @tf.function
  def call(self, x, intensity, training, mask):

    # adding embedding and position encoding.
    intensity = self.embedding(intensity)  # (batch_size, intensity_seq_len, d_model)
    x += intensity

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)


class Decoder2(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, dropout_rate=0.1):

    super(Decoder2, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, dropout_rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
  
  @tf.function
  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):

    attention_weights = {}

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

class ModifiedTransformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff,
               intensity_vocab_size, target_vocab_size, dropout_rate=0.1):
      super(ModifiedTransformer, self).__init__()

      self.encoder = Encoder2(num_layers, d_model, num_heads, dff,
                              intensity_vocab_size, dropout_rate)

      self.decoder = Decoder2(num_layers, d_model, num_heads,
                              dff, dropout_rate)

      self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, input, intensity, target, training, enc_padding_mask,
           look_ahead_mask, dec_padding_mask):
      enc_output = self.encoder(input, intensity, training, enc_padding_mask)  # (batch_size, input_seq_len, d_model)

      # dec_output.shape == (batch_size, tar_seq_len, d_model)
      dec_output, attention_weights = self.decoder(target, 
                                                   enc_output,
                                                   training,
                                                   look_ahead_mask,
                                                   dec_padding_mask)

      final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

      return final_output, attention_weights

In [ ]:
def create_masks(input, target):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(input)
    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(input)
    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(target)[1])
    dec_target_padding_mask = create_padding_mask(target)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=2))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')


'''
d_model : input(embedding), ouput 차원
num_layers : 인코더, 디코더 층
num_heads : 멀티헤드 수
d_ff : feedforward 차원 
'''
D_MODEL = 128
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT_RATE = 0.2

learning_rate = CustomSchedule(D_MODEL)
optimizer = tf.keras.optimizers.Adam(learning_rate,
                                     beta_1=0.9,
                                     beta_2=0.98,
                                     epsilon=1e-9)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [ ]:
def evaluate_aminoacid_level_finetuning(dataset):
    batch_size = 32
    num_batchs = 0
    accuracy = 0
    loss = 0

    dataset_batchs = dataset.padded_batch(batch_size = batch_size, drop_remainder=True)

    for batch, (input, intensity, target) in enumerate(dataset_batchs):
        num_batchs = batch+1
        target_input = target[:, :-1]
        target_real = target[:, 1:]
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(input, target_input)

        encoder1_output, decoder1_output, _, _ = \
            pretrained_transformer(input,
                                   target_input,
                                   False,
                                   enc_padding_mask,
                                   combined_mask,
                                   dec_padding_mask)

        predictions, _ = modified_transformer(encoder1_output,
                                              intensity,
                                              decoder1_output,
                                              False,
                                              enc_padding_mask,
                                              combined_mask,
                                              dec_padding_mask)

        loss += loss_function(target_real, predictions)
        accuracy += accuracy_function(target_real, predictions)

    return loss/num_batchs, accuracy/num_batchs

In [ ]:
pretrained_transformer = Transformer(
    num_layers=NUM_LAYERS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dff=DFF,
    input_vocab_size=600000,
    target_vocab_size=30,
    positional_encoding_input = 2000,
    positional_encoding_target = 50,
    dropout_rate=DROPOUT_RATE)

modified_transformer = ModifiedTransformer(
    num_layers=NUM_LAYERS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dff=DFF,
    intensity_vocab_size=12000,
    target_vocab_size=30,
    dropout_rate=DROPOUT_RATE)


In [ ]:
#load pretraining checkpoint

ckpt_path_pretraining = '/content/drive/MyDrive/translateMS/checkpoints/pretraining_128_2_8_512_0.2/ckpt-16'

ckpt_pretraining = tf.train.Checkpoint(transformer=pretrained_transformer,
                           optimizer=optimizer)

ckpt_pretraining.restore(ckpt_path_pretraining)
print('Checkpoint restored!!')

# if a checkpoint exists, restore the latest checkpoint.
#if ckpt_manager.latest_checkpoint:
#    ckpt.restore(ckpt_manager.latest_checkpoint)
#    print('Latest checkpoint restored!!')

Checkpoint restored!!


In [ ]:
checkpoint_path_finetuning = "/content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2"

ckpt_finetuning = tf.train.Checkpoint(transformer=modified_transformer,
                           optimizer=optimizer)

ckpt_manager_finetuning = tf.train.CheckpointManager(ckpt_finetuning, checkpoint_path_finetuning, max_to_keep=10)

#ckpt_finetuning.restore(checkpoint_path_finetuning)

if ckpt_manager_finetuning.latest_checkpoint:
    print(ckpt_manager_finetuning.latest_checkpoint)
    ckpt_finetuning.restore(ckpt_manager_finetuning.latest_checkpoint)
    print('Latest checkpoint restored!!')


/content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-51
Latest checkpoint restored!!


In [ ]:
train_step_signature_finetuning = [
  tf.TensorSpec(shape=(None, None), dtype=tf.int64),
  tf.TensorSpec(shape=(None, None), dtype=tf.int64),
  tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]

@tf.function(input_signature=train_step_signature_finetuning)
def train_step_finetuning(input, intensity, target):

    target_input = target[:, :-1]
    target_real = target[:, 1:]
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(input, target_input)

    with tf.GradientTape() as tape:
        #return : enc_output, dec_output, final_output, attention_weights
        encoder1_output, decoder1_output, _, _ = \
            pretrained_transformer(input,
                                   target_input,
                                   False,
                                   enc_padding_mask,
                                   combined_mask,
                                   dec_padding_mask)

        predictions, _ = modified_transformer(encoder1_output,
                                             intensity,
                                             decoder1_output,
                                             True,
                                             enc_padding_mask,
                                             combined_mask,
                                             dec_padding_mask)

        loss = loss_function(target_real, predictions)

    gradients = tape.gradient(loss, modified_transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, modified_transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(accuracy_function(target_real, predictions))

In [ ]:
feature_description = {
            'sequence': tf.io.VarLenFeature(tf.int64),
            'intensity': tf.io.VarLenFeature(tf.int64),
            'mz': tf.io.VarLenFeature(tf.int64),
            }

def parse_function(example_proto):
    parsed_example = tf.io.parse_single_example(example_proto,feature_description)
    mz = parsed_example['mz'].values
    intensity = parsed_example['intensity'].values
    sequence = parsed_example['sequence'].values
    return mz, intensity, sequence

path_real_train_dataset='/content/drive/MyDrive/translateMS/data/real_preprocessed_train_data.tfrecords'
path_real_valid_dataset='/content/drive/MyDrive/translateMS/data/real_preprocessed_valid_data.tfrecords'
path_real_test_dataset='/content/drive/MyDrive/translateMS/data/real_preprocessed_test_data.tfrecords'


size_dataset = 0

'''
number of data : 6503421
number of  data/real_preprocessed_train_data  :  3041294
number of  data/real_preprocessed_valid_data  :  380190
number of  data/real_preprocessed_test_data  :  390013

number of peptide in train dataset : 85058
number of peptide in valid dataset : 10660
number of peptide in test dataset : 10684
number of total peptide : 106402

'''
#train데이터는 절반 사용
real_train_dataset = tf.data.TFRecordDataset(path_real_train_dataset).map(parse_function).shard(num_shards=2, index=0)

#valid, test data는 1/10 사용
real_valid_dataset = tf.data.TFRecordDataset(path_real_valid_dataset).map(parse_function).shard(num_shards=10, index=0)
real_test_dataset = tf.data.TFRecordDataset(path_real_test_dataset).map(parse_function).shard(num_shards=10, index=0)

In [ ]:
cnt=0
for i in real_valid_dataset:
  cnt+=1
print(cnt)

KeyboardInterrupt: ignored

In [ ]:
BATCH_SIZE = 48
size_train_dataset = 3041294//2
NUM_BATCHS = int(size_train_dataset/BATCH_SIZE)
real_train_batchs = (real_train_dataset
                     .shuffle(4000000)
                     .padded_batch(BATCH_SIZE)
                     .prefetch(tf.data.AUTOTUNE))

In [ ]:
valid_loss, valid_accuracy = evaluate_aminoacid_level_finetuning(real_valid_dataset.take(64))
print(f'\tValid | Loss {valid_loss:.3f}, Accuracy {valid_accuracy:.3f}')

	Valid | Loss 0.722, Accuracy 0.804


In [ ]:
import time

epoch = 0

while True:
    start = time.time()
    train_loss.reset_states()
    train_accuracy.reset_states()

    for batch, (input, intensity, target) in enumerate(real_train_batchs):
        train_step_finetuning(input, intensity, target)

        print('\r',f'Epoch {epoch + 1} | batch {batch+1}/{NUM_BATCHS} Loss {train_loss.result():.3f} Accuracy {train_accuracy.result():.4f}',end='')

    print('\r',f'Epoch {epoch + 1} : Time {time.time() - start:.2f}s')
  
    print(f'\tTrain | Loss {train_loss.result():.3f}, Accuracy {train_accuracy.result():.3f}')
    valid_loss, valid_accuracy = evaluate_aminoacid_level_finetuning(real_valid_dataset)
    print(f'\tValid | Loss {valid_loss:.3f}, Accuracy {valid_accuracy:.3f}')

    ckpt_path_finetuning = ckpt_manager_finetuning.save()
    print('\r', f'Saving checkpoint for epoch {epoch + 1} at {ckpt_path_finetuning}')

    epoch+=1

 Epoch 1 | batch 22823/31680 Loss 1.335 Accuracy 0.5798

In [ ]:
'''
Epoch 1 : Time 6657.42s
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-1
	Train | Loss 2.242, Accuracy 0.291
	Valid | Loss 1.883, Accuracy 0.392
 Epoch 2 : Time 6649.44s
 Saving checkpoint for epoch 2 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-2
	Train | Loss 1.961, Accuracy 0.371
	Valid | Loss 1.874, Accuracy 0.399
 Epoch 3 : Time 6645.95s
 Saving checkpoint for epoch 3 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-3
	Train | Loss 1.834, Accuracy 0.414
	Valid | Loss 1.650, Accuracy 0.472
 Epoch 4 : Time 6598.67s
 Saving checkpoint for epoch 4 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-4
	Train | Loss 1.757, Accuracy 0.440
	Valid | Loss 1.648, Accuracy 0.477
 Epoch 1 : Time 6608.44s
	Train | Loss 1.704, Accuracy 0.458
	Valid | Loss 1.616, Accuracy 0.490
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-5
 Epoch 2 : Time 6565.93s
	Train | Loss 1.664, Accuracy 0.472
	Valid | Loss 1.615, Accuracy 0.493
 Saving checkpoint for epoch 2 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-6
 Epoch 3 : Time 6557.52s
	Train | Loss 1.632, Accuracy 0.483
	Valid | Loss 1.501, Accuracy 0.532
 Saving checkpoint for epoch 3 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-7
 Epoch 4 : Time 6565.39s
	Train | Loss 1.605, Accuracy 0.492
	Valid | Loss 1.446, Accuracy 0.548
 Saving checkpoint for epoch 4 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-8
 Epoch 5 : Time 6534.36s
	Train | Loss 1.582, Accuracy 0.500
	Valid | Loss 1.449, Accuracy 0.550
 Saving checkpoint for epoch 5 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-9
 Epoch 6 : Time 6530.70s
	Train | Loss 1.562, Accuracy 0.506
	Valid | Loss 1.379, Accuracy 0.573
 Saving checkpoint for epoch 6 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-10
 Epoch 7 : Time 6542.19s
	Train | Loss 1.545, Accuracy 0.512
	Valid | Loss 1.474, Accuracy 0.542
 Saving checkpoint for epoch 7 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-11
  Epoch 1 : Time 6350.71s
	Train | Loss 1.517, Accuracy 0.521
	Valid | Loss 1.408, Accuracy 0.566
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-13
 Epoch 2 : Time 6336.78s
	Train | Loss 1.504, Accuracy 0.526
	Valid | Loss 1.303, Accuracy 0.598
 Saving checkpoint for epoch 2 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-14
 Epoch 3 : Time 6338.12s
	Train | Loss 1.493, Accuracy 0.529
	Valid | Loss 1.341, Accuracy 0.586
 Saving checkpoint for epoch 3 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-15
 Epoch 4 : Time 6334.88s
	Train | Loss 1.482, Accuracy 0.532
	Valid | Loss 1.326, Accuracy 0.591
 Saving checkpoint for epoch 4 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-16
 Epoch 5 : Time 6333.28s
	Train | Loss 1.473, Accuracy 0.535
	Valid | Loss 1.338, Accuracy 0.588
 Saving checkpoint for epoch 5 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-17
 Epoch 6 : Time 6334.97s
	Train | Loss 1.465, Accuracy 0.538
	Valid | Loss 1.295, Accuracy 0.599
 Saving checkpoint for epoch 6 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-18
 Epoch 7 : Time 6334.57s
	Train | Loss 1.456, Accuracy 0.541
	Valid | Loss 1.299, Accuracy 0.599
 Saving checkpoint for epoch 7 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-19
 Epoch 8 : Time 6341.37s
	Train | Loss 1.449, Accuracy 0.543
	Valid | Loss 1.268, Accuracy 0.611
 Saving checkpoint for epoch 8 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-20
  Epoch 1 : Time 6422.34s
	Train | Loss 1.442, Accuracy 0.545
	Valid | Loss 1.279, Accuracy 0.607
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-21
 Epoch 2 : Time 6413.57s
	Train | Loss 1.435, Accuracy 0.548
	Valid | Loss 1.305, Accuracy 0.601
 Saving checkpoint for epoch 2 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-22
 Epoch 3 : Time 6402.30s
	Train | Loss 1.429, Accuracy 0.549
	Valid | Loss 1.244, Accuracy 0.620
 Saving checkpoint for epoch 3 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-23
 Epoch 4 : Time 6421.73s
	Train | Loss 1.424, Accuracy 0.551
	Valid | Loss 1.282, Accuracy 0.606
 Saving checkpoint for epoch 4 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-24
  Epoch 1 : Time 6387.95s
	Train | Loss 1.419, Accuracy 0.553
	Valid | Loss 1.237, Accuracy 0.617
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-25
 Epoch 2 : Time 6356.98s
	Train | Loss 1.413, Accuracy 0.555
	Valid | Loss 1.224, Accuracy 0.624
 Saving checkpoint for epoch 2 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-26
 Epoch 3 : Time 6358.52s
	Train | Loss 1.408, Accuracy 0.556
	Valid | Loss 1.304, Accuracy 0.599
 Saving checkpoint for epoch 3 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-27
 Epoch 4 : Time 6359.08s
	Train | Loss 1.404, Accuracy 0.558
	Valid | Loss 1.220, Accuracy 0.624
 Saving checkpoint for epoch 4 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-28
 Epoch 5 : Time 6392.57s
	Train | Loss 1.400, Accuracy 0.559
	Valid | Loss 1.223, Accuracy 0.626
 Saving checkpoint for epoch 5 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-29
 Epoch 1 : Time 3425.67s
	Train | Loss 1.396, Accuracy 0.560
	Valid | Loss 1.265, Accuracy 0.614
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-30
 Epoch 2 : Time 3409.74s
	Train | Loss 1.391, Accuracy 0.562
	Valid | Loss 1.233, Accuracy 0.623
 Saving checkpoint for epoch 2 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-31
 Epoch 3 : Time 3410.26s
	Train | Loss 1.388, Accuracy 0.563
	Valid | Loss 1.250, Accuracy 0.617
 Saving checkpoint for epoch 3 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-32
 Epoch 4 : Time 3410.29s
	Train | Loss 1.384, Accuracy 0.564
	Valid | Loss 1.220, Accuracy 0.628
 Saving checkpoint for epoch 4 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-33
 Epoch 5 : Time 3407.71s
	Train | Loss 1.381, Accuracy 0.565
	Valid | Loss 1.194, Accuracy 0.635
 Saving checkpoint for epoch 5 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-34
 Epoch 6 : Time 3406.20s
	Train | Loss 1.377, Accuracy 0.566
	Valid | Loss 1.188, Accuracy 0.636
 Saving checkpoint for epoch 6 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-35
 Epoch 7 : Time 3409.95s
	Train | Loss 1.374, Accuracy 0.567
	Valid | Loss 1.219, Accuracy 0.630
 Saving checkpoint for epoch 7 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-36
 Epoch 8 : Time 3408.65s
	Train | Loss 1.371, Accuracy 0.568
	Valid | Loss 1.258, Accuracy 0.618
 Saving checkpoint for epoch 8 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-37
 Epoch 9 : Time 3411.49s
	Train | Loss 1.368, Accuracy 0.569
	Valid | Loss 1.184, Accuracy 0.640
 Saving checkpoint for epoch 9 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-38
 Epoch 10 : Time 3406.89s
	Train | Loss 1.365, Accuracy 0.570
	Valid | Loss 1.202, Accuracy 0.637
 Saving checkpoint for epoch 10 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-39
 Epoch 11 : Time 3398.39s
	Train | Loss 1.363, Accuracy 0.571
	Valid | Loss 1.235, Accuracy 0.623
 Saving checkpoint for epoch 11 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-40
 Epoch 12 : Time 3397.46s
	Train | Loss 1.359, Accuracy 0.572
	Valid | Loss 1.195, Accuracy 0.636
 Saving checkpoint for epoch 12 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-41
 Epoch 13 : Time 3405.18s
	Train | Loss 1.357, Accuracy 0.573
	Valid | Loss 1.197, Accuracy 0.636
 Saving checkpoint for epoch 13 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-42
 Epoch 14 : Time 3408.87s
	Train | Loss 1.354, Accuracy 0.574
	Valid | Loss 1.196, Accuracy 0.636
 Saving checkpoint for epoch 14 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-43
 Epoch 15 : Time 3406.78s
	Train | Loss 1.352, Accuracy 0.574
	Valid | Loss 1.183, Accuracy 0.640
 Saving checkpoint for epoch 15 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-44
 Epoch 16 : Time 3407.64s
	Train | Loss 1.350, Accuracy 0.575
	Valid | Loss 1.199, Accuracy 0.634
 Saving checkpoint for epoch 16 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-45
 Epoch 17 : Time 3401.42s
	Train | Loss 1.348, Accuracy 0.576
	Valid | Loss 1.179, Accuracy 0.644
 Saving checkpoint for epoch 17 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-46
 Epoch 1 : Time 6491.51s
	Train | Loss 1.345, Accuracy 0.576
	Valid | Loss 1.173, Accuracy 0.642
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-47
  Epoch 1 : Time 6524.36s
	Train | Loss 1.344, Accuracy 0.577
	Valid | Loss 1.175, Accuracy 0.644
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-48
  Epoch 1 : Time 9660.41s
	Train | Loss 1.342, Accuracy 0.578
	Valid | Loss 1.181, Accuracy 0.641
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-49
 Epoch 1 : Time 6680.26s
	Train | Loss 1.339, Accuracy 0.578
	Valid | Loss 1.198, Accuracy 0.639
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-50
  Epoch 1 : Time 9959.75s
	Train | Loss 1.337, Accuracy 0.579
	Valid | Loss 1.186, Accuracy 0.639
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-51

'''

'''
Epoch 1 : Time 3387.09s
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-1
	Train | Loss 2.443, Accuracy 0.240
	Valid | Loss 2.260, Accuracy 0.293
 Epoch 2 : Time 3382.02s
 Saving checkpoint for epoch 2 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-2
	Train | Loss 2.228, Accuracy 0.305
	Valid | Loss 2.091, Accuracy 0.353
 Epoch 3 : Time 3381.14s
 Saving checkpoint for epoch 3 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-3
	Train | Loss 2.067, Accuracy 0.358
	Valid | Loss 1.946, Accuracy 0.401
 Epoch 4 : Time 3383.31s
 Saving checkpoint for epoch 4 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-4
	Train | Loss 1.941, Accuracy 0.398
	Valid | Loss 1.841, Accuracy 0.436
 Epoch 5 : Time 3379.02s
 Saving checkpoint for epoch 5 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-5
	Train | Loss 1.853, Accuracy 0.425
	Valid | Loss 1.772, Accuracy 0.459
 Epoch 6 : Time 3380.21s
 Saving checkpoint for epoch 6 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-6
	Train | Loss 1.785, Accuracy 0.446
	Valid | Loss 1.715, Accuracy 0.477
 Epoch 7 : Time 3377.84s
 Saving checkpoint for epoch 7 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-7
	Train | Loss 1.733, Accuracy 0.462
	Valid | Loss 1.666, Accuracy 0.490
 Epoch 8 : Time 3380.69s
 Saving checkpoint for epoch 8 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-8
	Train | Loss 1.692, Accuracy 0.474
	Valid | Loss 1.636, Accuracy 0.501
 Epoch 9 : Time 3383.05s
 Saving checkpoint for epoch 9 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-9
	Train | Loss 1.661, Accuracy 0.483
	Valid | Loss 1.606, Accuracy 0.510
 Epoch 10 : Time 3386.42s
 Saving checkpoint for epoch 10 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-10
	Train | Loss 1.632, Accuracy 0.492
	Valid | Loss 1.592, Accuracy 0.518
 Epoch 11 : Time 3380.77s
 Saving checkpoint for epoch 11 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-11
	Train | Loss 1.610, Accuracy 0.499
	Valid | Loss 1.578, Accuracy 0.521
 Epoch 12 : Time 3383.10s
 Saving checkpoint for epoch 12 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-12
	Train | Loss 1.590, Accuracy 0.504
	Valid | Loss 1.539, Accuracy 0.530
 Epoch 13 : Time 3383.12s
 Saving checkpoint for epoch 13 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-13
	Train | Loss 1.572, Accuracy 0.510
	Valid | Loss 1.525, Accuracy 0.534
 Epoch 14 : Time 3382.49s
 Saving checkpoint for epoch 14 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-14
	Train | Loss 1.557, Accuracy 0.514
	Valid | Loss 1.514, Accuracy 0.537
 Epoch 15 : Time 3383.06s
 Saving checkpoint for epoch 15 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-15
	Train | Loss 1.544, Accuracy 0.518
	Valid | Loss 1.505, Accuracy 0.539
 Epoch 16 : Time 3383.69s
 Saving checkpoint for epoch 16 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-16
	Train | Loss 1.533, Accuracy 0.521
	Valid | Loss 1.504, Accuracy 0.542
 Epoch 17 : Time 3389.24s
 Saving checkpoint for epoch 17 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-17
	Train | Loss 1.522, Accuracy 0.524
	Valid | Loss 1.500, Accuracy 0.542
 Epoch 18 : Time 3391.67s
 Saving checkpoint for epoch 18 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-18
	Train | Loss 1.512, Accuracy 0.528
	Valid | Loss 1.483, Accuracy 0.547
 Epoch 19 : Time 3388.94s
 Saving checkpoint for epoch 19 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-19
	Train | Loss 1.503, Accuracy 0.530
	Valid | Loss 1.467, Accuracy 0.549
 Epoch 20 : Time 3390.13s
 Saving checkpoint for epoch 20 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-20
	Train | Loss 1.495, Accuracy 0.533
	Valid | Loss 1.458, Accuracy 0.552
 Epoch 21 : Time 3384.88s
 Saving checkpoint for epoch 21 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-21
	Train | Loss 1.487, Accuracy 0.535
	Valid | Loss 1.471, Accuracy 0.552
 Epoch 22 : Time 3380.05s
 Saving checkpoint for epoch 22 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-22
	Train | Loss 1.480, Accuracy 0.537
	Valid | Loss 1.458, Accuracy 0.554
 Epoch 23 : Time 3368.96s
 Saving checkpoint for epoch 23 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-23
	Train | Loss 1.473, Accuracy 0.539
	Valid | Loss 1.459, Accuracy 0.555
 Epoch 24 : Time 3374.87s
 Saving checkpoint for epoch 24 at /content/drive/MyDrive/translateMS/checkpoints/finetuning_128_2_8_512_0.2/ckpt-24
	Train | Loss 1.467, Accuracy 0.541
	Valid | Loss 1.452, Accuracy 0.556
'''


'''
 Epoch 1 : Time 6050.81s
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-1
	Train | Loss 2.026, Accuracy 0.357
	Valid | Loss 1.549, Accuracy 0.515
 Epoch 2 : Time 6051.91s
 Saving checkpoint for epoch 2 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-2
	Train | Loss 1.637, Accuracy 0.483
	Valid | Loss 1.354, Accuracy 0.577
 Epoch 3 : Time 6041.49s
 Saving checkpoint for epoch 3 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-3
	Train | Loss 1.477, Accuracy 0.535
	Valid | Loss 1.260, Accuracy 0.613
 Epoch 4 : Time 5994.15s
 Saving checkpoint for epoch 4 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-4
	Train | Loss 1.380, Accuracy 0.567
	Valid | Loss 1.179, Accuracy 0.637
 Epoch 5 : Time 5911.29s
 Saving checkpoint for epoch 5 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-5
	Train | Loss 1.309, Accuracy 0.589
	Valid | Loss 1.138, Accuracy 0.648
 Epoch 6 : Time 5911.01s
 Saving checkpoint for epoch 6 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-6
	Train | Loss 1.255, Accuracy 0.606
	Valid | Loss 1.081, Accuracy 0.667
 Epoch 7 : Time 5902.05s
 Saving checkpoint for epoch 7 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-7
	Train | Loss 1.212, Accuracy 0.620
	Valid | Loss 1.085, Accuracy 0.664
 Epoch 8 : Time 5911.82s
 Saving checkpoint for epoch 8 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-8
	Train | Loss 1.177, Accuracy 0.630
	Valid | Loss 1.097, Accuracy 0.662
  Epoch 1 : Time 6030.05s
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-13
	Train | Loss 1.146, Accuracy 0.640
	Valid | Loss 1.094, Accuracy 0.667
 Epoch 2 : Time 6000.78s
 Saving checkpoint for epoch 2 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-14
	Train | Loss 1.117, Accuracy 0.650
	Valid | Loss 1.009, Accuracy 0.691
 Epoch 3 : Time 6004.16s
 Saving checkpoint for epoch 3 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-15
	Train | Loss 1.099, Accuracy 0.655
	Valid | Loss 0.969, Accuracy 0.702
 Epoch 4 : Time 6002.66s
 Saving checkpoint for epoch 4 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-16
	Train | Loss 1.085, Accuracy 0.660
	Valid | Loss 0.975, Accuracy 0.703
 Epoch 5 : Time 6002.51s
 Saving checkpoint for epoch 5 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-17
	Train | Loss 1.073, Accuracy 0.663
	Valid | Loss 0.918, Accuracy 0.719
 Epoch 6 : Time 5997.64s
 Saving checkpoint for epoch 6 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-18
	Train | Loss 1.062, Accuracy 0.667
	Valid | Loss 0.961, Accuracy 0.706
 Epoch 7 : Time 5998.73s
 Saving checkpoint for epoch 7 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-19
	Train | Loss 1.052, Accuracy 0.670
	Valid | Loss 0.948, Accuracy 0.711
 Epoch 8 : Time 5994.41s
 Saving checkpoint for epoch 8 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-20
	Train | Loss 1.044, Accuracy 0.673
	Valid | Loss 0.946, Accuracy 0.710
  Epoch 9 : Time 5996.41s
 Saving checkpoint for epoch 9 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-21
	Train | Loss 1.035, Accuracy 0.675
	Valid | Loss 0.898, Accuracy 0.729
 Epoch 10 : Time 5799.80s
 Saving checkpoint for epoch 10 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-22
	Train | Loss 1.028, Accuracy 0.677
	Valid | Loss 0.899, Accuracy 0.726
 Epoch 11 : Time 5789.37s
 Saving checkpoint for epoch 11 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-23
	Train | Loss 1.021, Accuracy 0.680
	Valid | Loss 0.888, Accuracy 0.729
 Epoch 12 : Time 5780.74s
 Saving checkpoint for epoch 12 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-24
	Train | Loss 1.015, Accuracy 0.682
	Valid | Loss 0.865, Accuracy 0.736
 Epoch 13 : Time 5769.57s
 Saving checkpoint for epoch 13 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-25
	Train | Loss 1.009, Accuracy 0.684
	Valid | Loss 0.881, Accuracy 0.729
 Epoch 14 : Time 5814.95s
 Saving checkpoint for epoch 14 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-26
	Train | Loss 1.003, Accuracy 0.685
	Valid | Loss 0.891, Accuracy 0.730
  Epoch 1 : Time 10898.10s
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-27
	Train | Loss 0.998, Accuracy 0.687
 	Valid | Loss 0.876, Accuracy 0.733
 Epoch 2 : Time 10833.03s
 Saving checkpoint for epoch 2 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-28
	Train | Loss 0.993, Accuracy 0.689
	Valid | Loss 0.860, Accuracy 0.738
 Epoch 3 : Time 10826.93s
 Saving checkpoint for epoch 3 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-29
	Train | Loss 0.988, Accuracy 0.690
	Valid | Loss 0.849, Accuracy 0.739
 Epoch 4 : Time 10845.81s
 Saving checkpoint for epoch 4 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-30
	Train | Loss 0.983, Accuracy 0.692
	Valid | Loss 0.849, Accuracy 0.742
 Epoch 5 : Time 10851.41s
 Saving checkpoint for epoch 5 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-31
	Train | Loss 0.979, Accuracy 0.693
	Valid | Loss 0.862, Accuracy 0.736
 Epoch 6 : Time 10857.52s
 Saving checkpoint for epoch 6 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-32
	Train | Loss 0.976, Accuracy 0.694
	Valid | Loss 0.875, Accuracy 0.730
 Epoch 7 : Time 10869.74s
 Saving checkpoint for epoch 7 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-33
	Train | Loss 0.972, Accuracy 0.695
	Valid | Loss 0.859, Accuracy 0.739
   Epoch 1 : Time 5749.89s
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-34
	Train | Loss 0.968, Accuracy 0.696
  Valid | Loss 0.833, Accuracy 0.748
 Epoch 1 : Time 5728.08s
 Saving checkpoint for epoch 1 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-36
	Train | Loss 0.965, Accuracy 0.698
	Valid | Loss 0.831, Accuracy 0.749
 Epoch 2 : Time 5725.86s
 Saving checkpoint for epoch 2 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-37
	Train | Loss 0.962, Accuracy 0.699
	Valid | Loss 0.876, Accuracy 0.735
 Epoch 3 : Time 5735.52s
 Saving checkpoint for epoch 3 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-38
	Train | Loss 0.958, Accuracy 0.700
	Valid | Loss 0.827, Accuracy 0.749
 Epoch 4 : Time 5721.86s
 Saving checkpoint for epoch 4 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-39
	Train | Loss 0.955, Accuracy 0.701
	Valid | Loss 0.820, Accuracy 0.751
 Epoch 5 : Time 5723.23s
 Saving checkpoint for epoch 5 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-40
	Train | Loss 0.953, Accuracy 0.702
	Valid | Loss 0.853, Accuracy 0.744
 Epoch 6 : Time 5722.20s
 Saving checkpoint for epoch 6 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-41
	Train | Loss 0.950, Accuracy 0.702
	Valid | Loss 0.819, Accuracy 0.752
 Epoch 7 : Time 5732.16s
 Saving checkpoint for epoch 7 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-42
	Train | Loss 0.947, Accuracy 0.703
	Valid | Loss 0.829, Accuracy 0.749
 Epoch 8 : Time 5724.54s
 Saving checkpoint for epoch 8 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-43
	Train | Loss 0.944, Accuracy 0.704
	Valid | Loss 0.831, Accuracy 0.750
 Epoch 9 : Time 5733.83s
 Saving checkpoint for epoch 9 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-44
	Train | Loss 0.942, Accuracy 0.705
	Valid | Loss 0.832, Accuracy 0.750
 Epoch 10 : Time 5723.48s
 Saving checkpoint for epoch 10 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-45
	Train | Loss 0.940, Accuracy 0.706
	Valid | Loss 0.815, Accuracy 0.756
 Epoch 11 : Time 5728.75s
 Saving checkpoint for epoch 11 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-46
	Train | Loss 0.937, Accuracy 0.707
	Valid | Loss 0.800, Accuracy 0.757
 Epoch 12 : Time 5763.87s
 Saving checkpoint for epoch 12 at /content/drive/MyDrive/translateMS/checkpoints/finetuning/ckpt-47
	Train | Loss 0.935, Accuracy 0.707
	Valid | Loss 0.812, Accuracy 0.753  
'''

In [ ]:
def evaluate_peptide_level(dataset, max_length = 50):
    cnt_total =0
    cnt_correct = 0
    for mz, intensity, sequence in dataset:
        cnt_total+=1
        if(cnt_total%10 == 0):
            print(cnt_total, cnt_correct/(cnt_total-1))

        encoder_input = tf.convert_to_tensor([mz])
        start, end = 1,2
        output = tf.convert_to_tensor([start],dtype=tf.int64)
        output = tf.expand_dims(output, 0)

        for i in range(max_length):
            enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
                encoder_input, output)
            # predictions.shape == (batch_size, seq_len, vocab_size)
            
            
            encoder1_output, decoder1_output, _, _ = \
                pretrained_transformer(encoder_input,
                                      output,
                                      False,
                                      enc_padding_mask,
                                      combined_mask,
                                      dec_padding_mask)

            predictions, _ = modified_transformer(encoder1_output,
                                                  intensity,
                                                  decoder1_output,
                                                  False,
                                                  enc_padding_mask,
                                                  combined_mask,
                                                  dec_padding_mask)

             
            # select the last word from the seq_len dimension
            predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

            predicted_id = tf.argmax(predictions, axis=-1)

            # concatentate the predicted_id to the output which is given to the decoder
            # as its input.
            output = tf.concat([output, predicted_id], axis=-1)

            # return the result if the predicted_id is equal to the end token
            if predicted_id == end:
                if output.shape[1]==sequence.shape[0] and tf.reduce_all(output[0] == sequence):
                    cnt_correct+=1
                break

    return cnt_correct/cnt_total

In [ ]:
test_loss, test_accuracy = evaluate_aminoacid_level_finetuning(real_test_dataset.shuffle(40000).take(5000))
print(f'\ttest | Loss {test_loss:.3f}, Accuracy {test_accuracy:.3f}')

In [ ]:

print(f'\ttest | Loss {test_loss:.3f}, Accuracy {test_accuracy:.3f}')

In [ ]:
accuracy_test_data = evaluate_peptide_level(real_test_dataset.shuffle(40000).take(2000))
print(f'Accuracy of test data for peptide level : {accuracy_test_data:.4f}')